In [4]:
import pandas as pd
import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output
import numpy as np

# Load your data
df_radar = pd.read_csv("CleanedData/player_radar.csv")
df_stats = pd.read_csv("CleanedData/player_stats_cleaned.csv")

# Merge dataframes on 'player' after normalizing the 'player' column
df_merged = pd.merge(df_radar, df_stats, on='player', how='outer')

# Calculate performance metrics
df_merged['passing_commulative_performance'] = (
    0.05*df_merged['passes_completed'] +
    1.5*df_merged['assists_x'] +
    1.5*df_merged['assisted_shots'] +
    0.2*df_merged['passes_into_final_third'] +
    0.4*df_merged['passes_into_penalty_area'] +
    0.3*df_merged['crosses_into_penalty_area'] +
    0.1*df_merged['progressive_passes']
)

df_merged['shooting_commulative_performance'] = (
    3*df_merged['goals_x'] +
    0.3*df_merged['shots_on_target'] +
    0.1*df_merged['shots'] +
    0.2*df_merged['shots_free_kicks'] +
    df_merged['pens_made_x']
)

df_merged['defence_commulative_performance'] = (
    -0.2*df_merged['dribbled_past'] -
    df_merged['errors'] +
    0.5*df_merged['blocks'] +
    0.75*df_merged['blocked_shots'] +
    0.75*df_merged['blocked_passes'] +
    df_merged['tackles_interceptions'] +
    0.2*df_merged['clearances']
)

df_merged['misc_commulative_performance'] = (
    -0.25*df_merged['cards_yellow'] -
    1*df_merged['cards_red'] -
    0.1*df_merged['fouls'] +
    0.25*df_merged['interceptions_x'] +
    0.3*df_merged['tackles_won_x'] +
    1.5*df_merged['pens_won'] -
    1.5*df_merged['pens_conceded'] +
    0.25*df_merged['ball_recoveries'] -
    3*df_merged['own_goals'] -
    0.25*df_merged['aerials_lost'] +
    0.25*df_merged['aerials_won']
)

app = Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='team-dropdown',
        options=[{'label': team, 'value': team} for team in df_merged['team_x'].dropna().unique()],
        placeholder="Select a team",
        clearable=False
    ),
    dcc.Dropdown(
        id='player-dropdown',
        placeholder="Select a player",
        clearable=False
    ),
    dcc.Graph(id='radar-chart', style={"height": "90vh"})  # Adjust the height of the radar chart
])

@app.callback(
    Output('player-dropdown', 'options'),
    Input('team-dropdown', 'value')
)
def set_player_options(selected_team):
    filtered_df = df_merged[df_merged['team_x'] == selected_team]
    return [{'label': player, 'value': player} for player in filtered_df['player'].unique()]

@app.callback(
    Output('radar-chart', 'figure'),
    [Input('team-dropdown', 'value'),
     Input('player-dropdown', 'value')]
)
def update_radar_chart(selected_team, selected_player):
    if not selected_team:
        return go.Figure()

    filtered_df = df_merged[df_merged['team_x'] == selected_team]
    categories = [
        'defence_commulative_performance',
        'passing_commulative_performance',
        'shooting_commulative_performance',
        'misc_commulative_performance',
        'defence_commulative_performance'  # Repeat the first category to close the radar chart loop
    ]
    categories_labels = ['Defence', 'Passing', 'Shooting', 'Miscellaneous', 'Defence']

    radial_ticks = np.arange(0, 60, 2)  # Adjust tick values 

    fig = go.Figure()

    for player in filtered_df['player'].unique():
        player_data = filtered_df[filtered_df['player'] == player]
        fig.add_trace(go.Scatterpolar(
            r=[player_data[cat].values[0] for cat in categories],
            theta=categories_labels,
            name=player,
            mode='lines',
            line_color='grey' if player != selected_player else 'green',
            line_shape='spline',  # Smoothing the lines
            line_smoothing=0.8,  # Adjust the smoothness
            opacity=0.65 if player != selected_player else 1,  # Highlight selected player
            line_width=1 if player != selected_player else 3  # Thicker line for selected player
        ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, tickvals=radial_ticks),
            bgcolor='white',
            radialaxis_tickfont_color='darkgrey',
            angularaxis_color='grey',
            angularaxis_showline=False,
            radialaxis_showline=False,
            radialaxis_gridcolor='#F2F2F2'
        ),
        title=f"Performance Metrics for {selected_team}",
        font=dict(size=15)
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)


In [2]:
import threading
from IPython.display import display, HTML

# Function to run the Dash app
def run_dash():
    app.run_server(debug=False, use_reloader=False)

# Start the Dash app in a separate thread
threading.Thread(target=run_dash).start()

# Display the link to open the app in the browser
display(HTML(f"""
    <a href="http://127.0.0.1:8051/" target="_blank">
        Open Dash App
    </a>
"""))